# Pytorch Model 

this notebook will demonstrate the same techniques and practices used in Emotion_Classifier notebook but using Pytorch instead of Tensorflow. Pytorch comes with enough tools that we don't need Librosa as well and Pytorch's audio module "Torchaudio" has audio tranformers that are based on Librosa's fundementals. Then biggest difference might be that Pytorch will turn an audiofile into a Tensor instead of a Numpy array like with Librosa. 

In [4]:
# Importing Necessary Libraries 

# Standard Python Operations 
import pandas as pd 
import numpy as np
import math
import os
import seaborn as sns
import matplotlib.pyplot as plt

# Audio and torch data
from scipy import signal 
import torchaudio 
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(0)


# Neural Network 
import torch 
import torch.nn as nn 



# Preparing data 
First the data will be prepared the same way as in my other notebooks. A Pandas Dataframe will be created that contains the Path of the audio file and its respected class label for each observation. Then I will create a custom Data Class object using Pytorch's Data Class

In [5]:

# Setting string representing path for folder containing audio files
Ravdess = '/Users/stephen/Emotion_Detection-/data/RAVDESS/'

# Turn Path string into Path Object 
ravdess_directory_list = os.listdir(Ravdess)
ravdess_directory_list.sort

# Create empty lists to store audio paths and their labels associated with each file 
file_emotion = []
file_path = []
# Since each actor is in there own folder we must iterate through each folder 
for dir in ravdess_directory_list:
    # as their are 24 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(Ravdess + dir)
    for file in actor:
        # splitting up file name to decode labels 
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(Ravdess + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()

,Emotions,Path
0,angry,/Users/stephen/Emotion_Detection-/data/RAVDESS...
1,fear,/Users/stephen/Emotion_Detection-/data/RAVDESS...
2,fear,/Users/stephen/Emotion_Detection-/data/RAVDESS...
3,angry,/Users/stephen/Emotion_Detection-/data/RAVDESS...
4,disgust,/Users/stephen/Emotion_Detection-/data/RAVDESS...


In [6]:
Ravdess_df.shape

(1440, 2)

In [ ]:
# Creating dataset object 

class Dataset(Dataset):

    # Constructor 
    def __init__(self, )